In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.manifold import TSNE
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [12]:
def feature_extract(df):
    # 説明変数とラベルの分離
    label_df = df['dengue']
    df = df.drop('dengue', axis=1)

    # もとのdfの正規化
    numerical_columns = df.columns.tolist()
    numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numerical_columns)])
    df_normalized = preprocessor.fit_transform(df)
    nomal_df= pd.DataFrame(df_normalized, columns=numerical_columns)  # カラム名を維持

    # 特徴量の格納用DataFrame
    features_df = pd.DataFrame()

    # クラスタリング
    kmeans = KMeans(n_clusters=4, random_state=0).fit(nomal_df)
    features_df['kmeans_dist'] = kmeans.transform(nomal_df).min(axis=1)
    features_df['kmeans_cluster'] = kmeans.labels_

    
    
    # 次元削減
    pca = PCA(n_components=4, random_state=0)
    pca_features = pca.fit_transform(nomal_df)
    features_df['pca_1'] = pca_features[:, 0]
    features_df['pca_2'] = pca_features[:, 1]
    features_df['pca_3'] = pca_features[:, 2]
    features_df['pca_4'] = pca_features[:, 3]

    tsne = TSNE(n_components=3, random_state=0, perplexity=30)
    tsne_features = tsne.fit_transform(nomal_df)
    features_df['tsne_1'] = tsne_features[:, 0]
    features_df['tsne_2'] = tsne_features[:, 1]
    features_df['tsne_3'] = tsne_features[:, 2]
    

    
    # 距離・類似度ベース
    cos_sim_matrix = cosine_similarity(nomal_df)
    nearest_distances = cos_sim_matrix.mean(axis=1)
    features_df['nearest_cosine_similarity'] = nearest_distances


    
    #学習用dfの標準化
    numerical_columns = features_df.columns.tolist()
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numerical_columns)])
    df_normalized = preprocessor.fit_transform(features_df)
    features_df = pd.DataFrame(df_normalized, columns=numerical_columns)  # カラム名を維持

    # 学習用データフレームの統合
    learning_df = pd.concat([df.reset_index(drop=True),features_df,label_df.reset_index(drop=True)], axis=1)


    return learning_df
    
    

In [13]:
df1=pd.read_csv('../data/preprocessed_data/df1.csv',index_col=0)
df2=pd.read_csv('../data/preprocessed_data/df2.csv',index_col=0)
df3=pd.read_csv('../data/preprocessed_data/df3.csv',index_col=0)
df4=pd.read_csv('../data/preprocessed_data/df4.csv',index_col=0)
df5=pd.read_csv('../data/preprocessed_data/df5.csv',index_col=0)




In [14]:
learning_df1=feature_extract(df1)
learning_df2=feature_extract(df2)


In [15]:
learning_df=pd.concat([learning_df1,learning_df2],axis=0)
learning_df.to_csv('../data/learning_data.csv')

In [16]:
learning_df

,fever,headache,muscle pain,joint pain,rash,nausea,vomiting,eye pain,abdominal pain,lymphadenopathy,...,kmeans_cluster,pca_1,pca_2,pca_3,pca_4,tsne_1,tsne_2,tsne_3,nearest_cosine_similarity,dengue
0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,-0.983576,1.220856,0.299660,-0.643981,-1.729037,0.516306,0.552540,-0.512182,-0.005099,1.0
1,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,-0.983576,0.644435,-0.771144,0.598800,1.043483,0.121641,-0.576519,0.603666,0.971852,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.537551,0.000511,1.165116,0.820401,-0.253762,-1.072705,-0.325399,-1.372784,0.204822,0.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,-0.983576,0.284339,-0.856532,1.591132,1.076701,-0.245241,-0.874049,1.036044,0.568445,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,-0.983576,0.835186,-0.957348,-0.077477,1.164248,1.948557,-1.575037,0.650907,0.302657,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,-1.182555,0.193072,0.682480,1.498404,0.873978,-0.759045,0.866572,-1.620569,0.414101,0.0
409,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,-0.288058,1.268619,-1.762018,-0.362751,-0.288538,0.572830,-1.185400,-1.741749,0.315529,0.0
410,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,-1.182555,0.136308,0.775940,1.740069,1.076107,-0.097876,0.102822,0.422938,0.142913,1.0
411,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.606439,-0.371947,-1.171813,1.305010,0.316745,-0.622541,-1.603136,-0.055569,0.538169,0.0
